<a href="https://colab.research.google.com/github/lesterrsantos/WebExample/blob/main/operaciones_basicas_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install pyspark==3.2.0

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 14.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8d6a81b2d74d7bc34477684a7bb0a689796209d6093b5cf9afc7be0683addfcb
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [5]:
import pyspark
from pyspark.sql import SparkSession, SQLContext
sc = SparkSession.builder.appName("OperacionesDatos").getOrCreate()

In [6]:
rdd=sc.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])
print(rdd.sum())
print(rdd.count())

45
9


In [ ]:
data=[('juan','Raul','Gonzalez','1979-04-29','M','B'),
      ('Carlos','','Perez','1973-02-02','M','M'),
      ('Berta','','Moreira','1985-04-21','F','B'),
      ('Julia','Elena','Martinez','1991-03-17','F','N'),
]
col=['Firstname', 'Middlename', 'Lastname', 'Day','Sex','Color']
df2=sc.createDataFrame(data=data, schema=col)
df2.show(4)

+---------+----------+--------+----------+---+-----+
|Firstname|Middlename|Lastname|       Day|Sex|Color|
+---------+----------+--------+----------+---+-----+
|     juan|      Raul|Gonzalez|1979-04-29|  M|    B|
|   Carlos|          |   Perez|1973-02-02|  M|    M|
|    Berta|          | Moreira|1985-04-21|  F|    B|
|    Julia|     Elena|Martinez|1991-03-17|  F|    N|
+---------+----------+--------+----------+---+-----+



In [ ]:
from pyspark.sql import functions as f

movies = sc.read.load("drive/MyDrive/imdb/ml-latest-small/movies.csv", format='csv',header=True,schema="movieId INT, title STRING, genres STRING",)
ratings = (sc.read.load("drive/MyDrive/imdb/ml-latest-small/ratings.csv", format='csv', header = True, schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
                       ).withColumnRenamed("timestamp", "timestamp_unix").withColumn("timestamp", f.to_timestamp(f.from_unixtime("timestamp_unix"))))
#links = sc.read.load("ml-latest-small/links.csv", format='csv', header = True)
#tags = sc.read.load("ml-latest-small/tags.csv", format='csv', header = True)


In [ ]:
movies.printSchema()
ratings.printSchema()
ratings.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp_unix: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+------+-------+------+--------------+-------------------+
|userId|movieId|rating|timestamp_unix|          timestamp|
+------+-------+------+--------------+-------------------+
|     1|     31|   2.5|    1260759144|2009-12-14 02:52:24|
|     1|   1029|   3.0|    1260759179|2009-12-14 02:52:59|
|     1|   1061|   3.0|    1260759182|2009-12-14 02:53:02|
|     1|   1129|   2.0|    1260759185|2009-12-14 02:53:05|
|     1|   1172|   4.0|    1260759205|2009-12-14 02:53:25|
+------+-------+------+--------------+-------------------+
only showing top 5 rows



In [ ]:
ratings.drop("timestamp_unix", "foobar").show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|     31|   2.5|2009-12-14 02:52:24|
|     1|   1029|   3.0|2009-12-14 02:52:59|
|     1|   1061|   3.0|2009-12-14 02:53:02|
|     1|   1129|   2.0|2009-12-14 02:53:05|
|     1|   1172|   4.0|2009-12-14 02:53:25|
+------+-------+------+-------------------+
only showing top 5 rows



In [ ]:
movies.where("genres == 'Action'").show(5)


+-------+--------------------+------+
|movieId|               title|genres|
+-------+--------------------+------+
|      9| Sudden Death (1995)|Action|
|     71|    Fair Game (1995)|Action|
|    204|Under Siege 2: Da...|Action|
|    251|  Hunted, The (1995)|Action|
|    667|Bloodsport 2 (a.k...|Action|
+-------+--------------------+------+
only showing top 5 rows



In [ ]:
movies.show(4)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows



In [ ]:
movie_genre = (
    movies
    .withColumn("genres_array", f.split("genres", "\|"))
    .withColumn("genre", f.explode("genres_array"))
    .select("movieId", "title","genres", "genre")
)
movies.show(10)
movie_genre.show(20)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows

+-------+--------------------+--------------------+---------+
|movieId|               title|              genres|    genre|
+-------+--------------------+--------------------+---------+
|      1|    Toy Story (1995)|Adventure|Animati...|Adventure

In [ ]:
dd=movie_genre.select("genre").distinct()
dd.show()
print(dd.count())
movies_without_genre = movies.where("genres == '(no genres listed)'")
movies_without_genre.show()
print(movies_without_genre.count())


+------------------+
|             genre|
+------------------+
|             Crime|
|           Romance|
|          Thriller|
|         Adventure|
|             Drama|
|               War|
|       Documentary|
|           Fantasy|
|           Mystery|
|           Musical|
|         Animation|
|         Film-Noir|
|(no genres listed)|
|              IMAX|
|            Horror|
|           Western|
|            Comedy|
|          Children|
|            Action|
|            Sci-Fi|
+------------------+

20
+-------+--------------------+------------------+
|movieId|               title|            genres|
+-------+--------------------+------------------+
|  83829|Scorpio Rising (1...|(no genres listed)|
| 117192|Doctor Who: The T...|(no genres listed)|
| 122888|      Ben-hur (2016)|(no genres listed)|
| 126106|Beastie Boys: Sab...|(no genres listed)|
| 128616|As We Were Dreami...|(no genres listed)|
| 128620|     Victoria (2015)|(no genres listed)|
| 129250|   Superfast! (2015)|(no genres l

In [ ]:
movie_genre.groupBy("genre").count().show()

+------------------+-----+
|             genre|count|
+------------------+-----+
|             Crime| 1100|
|           Romance| 1545|
|          Thriller| 1729|
|         Adventure| 1117|
|             Drama| 4365|
|               War|  367|
|       Documentary|  495|
|           Fantasy|  654|
|           Mystery|  543|
|           Musical|  394|
|         Animation|  447|
|         Film-Noir|  133|
|(no genres listed)|   18|
|              IMAX|  153|
|            Horror|  877|
|           Western|  168|
|            Comedy| 3315|
|          Children|  583|
|            Action| 1545|
|            Sci-Fi|  792|
+------------------+-----+



In [ ]:
sc.stop()